In [3]:
import librosa

path = "../data/input/train_short_audio/acafly/XC109605.ogg"
data, sample_rate = librosa.load(path)

In [4]:
data[:10], sample_rate

(array([-3.4471168e-07,  3.6376243e-06, -1.5128242e-06,  6.1439500e-06,
        -2.2385304e-05, -1.7057277e-05, -1.0762281e-05,  1.1271412e-05,
         5.1328945e-05, -3.3794720e-05], dtype=float32),
 22050)

In [5]:
# create data at 2 CENS/second

cens = librosa.feature.chroma_cens(data, sample_rate)
cens.shape

(12, 2743)

In [6]:
data.shape[0]/sample_rate

63.67460317460318

In [7]:
import numpy as np
sample_rate // ((np.arange(1,50) * (2 ** 6)))

array([344, 172, 114,  86,  68,  57,  49,  43,  38,  34,  31,  28,  26,
        24,  22,  21,  20,  19,  18,  17,  16,  15,  14,  14,  13,  13,
        12,  12,  11,  11,  11,  10,  10,  10,   9,   9,   9,   9,   8,
         8,   8,   8,   8,   7,   7,   7,   7,   7,   7], dtype=int32)

The hop length of CENS needs to be a factor of `2**6` (as per https://github.com/librosa/librosa/issues/658), so lets figure out how to handle this such that the number of CENS is betwen 2-10.

In [8]:
sample_rate//(2*(2**6))

172

In [9]:
def cens_per_sec(sample_rate, target):
    """Ensure this value is a multiple of 2**6"""
    return (sample_rate // (target * (2 ** 6))) * (2 ** 6)

In [10]:
cens = librosa.feature.chroma_cens(data, sample_rate, hop_length=cens_per_sec(sample_rate, 10))
cens.shape

(12, 646)

In [11]:
cens

array([[0.31563735, 0.31980844, 0.32348983, ..., 0.439828  , 0.44406454,
        0.44769552],
       [0.27548214, 0.27980544, 0.28418478, ..., 0.24253469, 0.24275353,
        0.2430237 ],
       [0.37096724, 0.36626072, 0.36172988, ..., 0.25960682, 0.26029104,
        0.26095763],
       ...,
       [0.35168645, 0.35078216, 0.34977211, ..., 0.24939341, 0.24826283,
        0.24730621],
       [0.35871711, 0.35795742, 0.35734785, ..., 0.32238248, 0.32164403,
        0.32120328],
       [0.26860333, 0.27073335, 0.27259668, ..., 0.36116633, 0.35543203,
        0.34967384]])

In [13]:
from pathlib import Path

cens_dir = Path("../data/cens")
output = cens_dir / Path(path).relative_to("../data/input").as_posix().replace(".ogg", ".npy")
output.parent.mkdir(exist_ok=True, parents=True)
np.save(output.as_posix(), cens)